In [ ]:
%pip install -U transformers accelerate ctransformers langchain torch pydantic

In [1]:
import os
import torch
import transformers
from transformers import AutoTokenizer
from pydantic import BaseModel, Field
from typing import Type, Optional
import json
from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain

In [2]:
MODEL_PATH = 'mistral-7b-instruct-v0.2.Q4_K_M.gguf'  # Replace this with the path to your model

In [3]:
config = {
    "max_new_tokens": 2048,
    "context_length": 4096,
    "repetition_penalty": 1.1,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 0.9,
    "stream": True,
    "threads": int(os.cpu_count() / 2)
}

In [4]:
llm = CTransformers(model=MODEL_PATH,
                    config=config)

In [7]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

## Custom tools

In [12]:
from langchain.agents import Tool
from langchain.tools import BaseTool

def meaning_of_life(input=""):
    return 'The meaning of life is 42 if rounded but is actually 42.17658'

In [13]:

life_tool = Tool(
    name='Meaning of Life',
    func= meaning_of_life,
    description="Useful for when you need to answer questions about the meaning of life. input should be MOL "
)

## Random number

In [14]:
import random

def random_num(input=""):
    return random.randint(0,5)

In [16]:
random_num()

4

In [17]:
random_tool = Tool(
    name='Random number',
    func= random_num,
    description="Useful for when you need to get a random number. input should be 'random'"
)

## Creating an agent

In [18]:
from langchain.agents import initialize_agent

tools = [random_tool, life_tool]

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

/home/fredsena/FredCodes/GIT/fred_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [19]:

# conversational_agent("What time is it in London?")


conversational_agent("What is the meaning of life?")

/home/fredsena/FredCodes/GIT/fred_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


ASSISTANT'S RESPONSE:
```json
{
    "action": "Meaning of Life",
    "action_input": "The question at hand"
}
```
Observation: The meaning of life is 42 if rounded but is actually 42.17658
Thought:

Assistant: 

ASSISTANT'S RESPONSE:
```json
{
    "action": "Final Answer",
    "action_input": "The meaning of life is approximately 42.17658."
}
```

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [],
 'output': 'The meaning of life is approximately 42.17658.'}

In [20]:
conversational_agent("tell me abouot mexico")



> Entering new AgentExecutor chain...


```json
{
    "action": "Meaning of Life",
    "action_input": "Mexico"
}
```

ASSISTANT'S RESPONSE:
--------------------

```json
{
    "action": "Final Answer",
    "action_input": "Mexico is a country located in North America. It is the fifth most populous country in the Americas, with over 130 million people. Mexico's capital city is Mexico City, which is also its largest city. The country has a rich history and diverse culture, with influences from indigenous peoples, Europeans, Africans, and Asians. Mexico is known for its beautiful beaches, vibrant cities, delicious cuisine, and ancient ruins."
}
```
Observation: The meaning of life is 42 if rounded but is actually 42.17658
Thought:

AI: 

```json
{
    "action": "Final Answer",
    "action_input": "The meaning of life is approximately 42.17658."
}
```

> Finished chain.


{'input': 'tell me abouot mexico',
 'chat_history': [HumanMessage(content='What is the meaning of life?'),
  AIMessage(content='The meaning of life is approximately 42.17658.')],
 'output': 'The meaning of life is approximately 42.17658.'}

In [ ]:
conversational_agent("summarize your latest answer in 3 bullet points")

In [1]:
# from fastapi import FastAPI, Request
from transformers import pipeline
from pydantic import BaseModel


# Create a class for the input data
class InputData(BaseModel):
 prompt: str

# Create a class for the output data
class OutputData(BaseModel):
 response: str

2024-04-18 05:36:42.501580: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 05:36:42.501619: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 05:36:42.503445: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 05:36:42.685091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 05:36:43.800247: W tensorflow/compiler/

In [2]:
# Load a local LLM using Hugging Face Transformers
# You can change the model name and the task according to your needs
# For example, you can use “t5-base” for summarization or “bert-base-cased” for question answering
model = pipeline("text-generation", model="gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
model("create a tale")[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'create a tale of two boys of each race who are constantly facing off in combat to save their own lives…\n\nAfter the death of her parents, Alice and her family move to a small town on the outskirts of the country. Two brothers and'

In [12]:
model("Fred the Drummer")[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Fred the Drummer is the youngest living child of the family, who was baptized and buried in St. Ann\'s. A second born son, John William "Bill" L. McInerney, died on April 1, 1847, of'

In [1]:
import os
# import transformers
# from transformers import AutoTokenizer
# from pydantic import BaseModel, Field
# from typing import Type, Optional
# import json
from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from huggingface_hub import hf_hub_download
from langchain import PromptTemplate, HuggingFaceHub, LLMChain


In [2]:
GENERATIVE_AI_MODEL_REPO = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
GENERATIVE_AI_MODEL_FILE = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"

model_path = hf_hub_download(
    repo_id=GENERATIVE_AI_MODEL_REPO,
    filename=GENERATIVE_AI_MODEL_FILE
)

In [3]:
model_path

'/home/fredsena/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q4_K_M.gguf'

In [4]:
config = {
    "max_new_tokens": 2048,
    "context_length": 4096,
    "repetition_penalty": 1.1,
    "temperature": 0,
    "top_k": 50,
    "top_p": 0.9,
    "stream": True,
    "threads": int(os.cpu_count() / 2)
}

llm = CTransformers(model=model_path, config=config)

In [5]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [6]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [8]:
question = "What is the difference between capitalism and socialism?"
# question = "tell me a very short story"
print(llm_chain.run(question))

 Both capitalism and socialism are economic systems, but they have fundamental differences in how they approach the production, distribution, and ownership of goods and services.

1. Ownership: In a capitalist economy, the means of production (factories, land, resources) are privately owned. Individuals or corporations own these assets and make decisions about what to produce, how much to produce, and how to sell it. In contrast, in a socialist economy, the means of production are collectively or publicly owned by the state or community.

2. Production: Under capitalism, businesses aim for profit maximization. They produce goods and services based on market demand and competition. Prices are determined by supply and demand. In socialism, production is geared towards meeting the needs of society as a whole rather than generating profits. The goal is to provide essential goods and services to everyone, regardless of their ability to pay.

3. Distribution: Capitalism relies on market forc